# Data Querying

The notebook runs some basic and advanced SQL queries to perform some bivariate analyses to understand customer spending behavior before training and evaluating prediction models. Amazon Athena was used to read SQL queries. 

## Importing libraries and initiating sagemaker session

In [2]:
#!pip install awswrangler

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import sagemaker
import boto3
import botocore
import awswrangler as wr

config = botocore.config.Config()
sm = boto3.client(service_name="sagemaker", config=config)
sess = sagemaker.Session(sagemaker_client=sm)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

In [26]:
wr.catalog.create_database(
    name='UK Online Retail Store Database',
    exist_ok=True
)


In [6]:
wr.catalog.create_csv_table(database = "UK Online Retail Store Database",
                           path = f"s3://{bucket}/data/input/",
                           table = "df_input",
                           columns_types = {"CustomerID": "float",
                                            "Country": "string",
                                            "Recency": "int",
                                            "Frequency": "int",
                                            "DailySpending": "float",
                                            "DailyTransCount":"float",
                                            "MonetaryValue_x": "float",
                                            "MonetaryValue_y": "float"},
                           mode = "overwrite",
                           skip_header_line_count = 1,
                           sep = ",")

In [7]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(region)))

In [10]:
wr.athena.create_athena_bucket()

's3://aws-athena-query-results-397738742408-us-east-2/'

## SQL Queries

#### Reviewing the data

In [27]:
sql_statment = """
SELECT *
FROM df_input
"""

df = wr.athena.read_sql_query(sql=sql_statment, database = "UK Online Retail Store Database")
df.head()

,customerid,country,recency,frequency,dailyspending,dailytranscount,monetaryvalue_x,monetaryvalue_y
0,13313.0,United Kingdom,53,31,304.869995,16.0,609.739990,945.580017
1,18097.0,United Kingdom,43,49,637.020020,24.0,1274.040039,1241.239990
2,16656.0,United Kingdom,30,27,625.744019,5.0,3128.719971,2729.825928
3,16875.0,United Kingdom,134,46,402.545013,23.0,805.090027,1290.439941
4,13094.0,United Kingdom,29,12,124.199997,2.0,869.400024,834.239990


#### What are the top 5 countries in terms of average monetary value for the first 6 months?

In [28]:
sql_statement = """
SELECT country, AVG(monetaryvalue_x) AS avg_monetary_value
FROM df_input
GROUP BY country
ORDER BY avg_monetary_value DESC
LIMIT 5
"""

wr.athena.read_sql_query(sql=sql_statement, database = "UK Online Retail Store Database")

,country,avg_monetary_value
0,Singapore,2427.295898
1,Norway,1665.692017
2,Portugal,1638.692261
3,EIRE,1634.269287
4,Japan,1504.767090


#### What are the average monetary values of the most recent (95th percentile) & least recent(5th percentile) customers?

In [34]:
sql_statement = """
WITH CTE1 AS (
    SELECT customerid, recency, monetaryvalue_x, monetaryvalue_y, NTILE(20) OVER(ORDER BY recency ASC) AS pct
    FROM df_input
    ),
CTE2 AS (
    SELECT 'Most Recent' AS customer_group, AVG(monetaryvalue_x) AS avg_monetary_value_1, AVG(monetaryvalue_y) AS avg_monetary_value_2
    FROM CTE1
    WHERE pct<=1
    ),
CTE3 AS (
    SELECT 'Least Recent' AS customer_group, AVG(monetaryvalue_x) AS avg_monetary_value_1, AVG(monetaryvalue_y) AS avg_monetary_value_2
    FROM CTE1
    WHERE pct>=19
    )
SELECT *
FROM CTE2
UNION ALL
SELECT *
FROM CTE3
"""

wr.athena.read_sql_query(sql=sql_statement, database = "UK Online Retail Store Database")

,customer_group,avg_monetary_value_1,avg_monetary_value_2
0,Least Recent,430.465729,717.267395
1,Most Recent,1242.645630,1399.087769


#### What are the average monetary values of the most frequent (95th percentile) & least frequent (5th percentile) customers?

In [35]:
sql_statement = """
WITH CTE1 AS (
    SELECT customerid, frequency, monetaryvalue_x, monetaryvalue_y, NTILE(20) OVER(ORDER BY frequency DESC) AS pct
    FROM df_input
    ),
CTE2 AS (
    SELECT 'Most Frequent' AS customer_group, AVG(monetaryvalue_x) AS avg_monetary_value_1, AVG(monetaryvalue_y) AS avg_monetary_value_2
    FROM CTE1
    WHERE pct<=1
    ),
CTE3 AS (
    SELECT 'Least Frequent' AS customer_group, AVG(monetaryvalue_x) AS avg_monetary_value_1, AVG(monetaryvalue_y) AS avg_monetary_value_2
    FROM CTE1
    WHERE pct>=19
    )
SELECT *
FROM CTE2
UNION ALL
SELECT *
FROM CTE3
"""

wr.athena.read_sql_query(sql=sql_statement, database = "UK Online Retail Store Database")

,customer_group,avg_monetary_value_1,avg_monetary_value_2
0,Least Frequent,308.197357,610.815796
1,Most Frequent,1775.693359,1761.506104


#### What are the average frequency & recency of the most valueable (95th percentile) & least (5th percentile) valueable customers?

In [40]:
sql_statement = """
WITH CTE1 AS (
    SELECT customerid, frequency, recency, monetaryvalue_x, NTILE(20) OVER(ORDER BY monetaryvalue_x DESC) AS pct
    FROM df_input
),
CTE2 AS (
    SELECT 'Most Valueable' AS customer_group, AVG(frequency) AS frequency, AVG(recency) AS recency
    FROM CTE1
    WHERE pct<=1
    ),
CTE3 AS (
    SELECT 'Least Valueable' AS customer_group, AVG(frequency) AS frequency, AVG(recency) AS recency
    FROM CTE1
    WHERE pct>=19
)
SELECT * 
FROM CTE2
UNION ALL
SELECT *
FROM CTE3
"""

wr.athena.read_sql_query(sql=sql_statement, database = "UK Online Retail Store Database")

,customer_group,frequency,recency
0,Most Valueable,91.723404,23.468085
1,Least Valueable,11.720430,61.817204


#### What are the average daily spending & transaction counts of the most valueable & least valueable customers?

In [41]:
sql_statement = """
WITH CTE1 AS (
    SELECT customerid, dailyspending, dailytranscount, monetaryvalue_x, NTILE(20) OVER(ORDER BY monetaryvalue_x DESC) AS pct
    FROM df_input
),
CTE2 AS (
    SELECT 'Most Valueable' AS customer_group, AVG(dailyspending) AS dailyspending, AVG(dailytranscount) AS dailytranscount
    FROM CTE1
    WHERE pct<=1
    ),
CTE3 AS (
    SELECT 'Least Valueable' AS customer_group, AVG(dailyspending) AS dailyspending, AVG(dailytranscount) AS dailytranscount
    FROM CTE1
    WHERE pct>=19
)
SELECT * 
FROM CTE2
UNION ALL
SELECT *
FROM CTE3
"""

wr.athena.read_sql_query(sql=sql_statement, database = "UK Online Retail Store Database")

,customer_group,dailyspending,dailytranscount
0,Most Valueable,479.856140,21.182980
1,Least Valueable,117.224457,11.309677


#### What is the average difference in monetary values for the first and last 6 months of 2011?

In [43]:
sql_statement = """
SELECT AVG(ABS(monetaryvalue_x - monetaryvalue_y)) AS avg_monetary_value_diff
FROM df_input
"""

wr.athena.read_sql_query(sql=sql_statement, database = "UK Online Retail Store Database")

,avg_monetary_value_diff
0,512.92218
